In [6]:
from datetime import datetime
from datetime import timedelta
from sportsreference.ncaab.boxscore import Boxscores
from sportsreference.ncaab.teams import Teams
import pandas as pd
import csv

In [7]:
def get_today():
    today = datetime.today()
    year = today.year
    month = today.month
    day = today.day
    return year,month,day

def get_yesterday():
    today = datetime.today()
    yesterday = today - timedelta(days = 1)
    year = yesterday.year
    month = yesterday.month
    day = yesterday.day
    return year,month,day

def get_scores(year, month ,day):
    home_teams = []
    away_teams = []
    home_scores = []
    away_scores = []
    
    games = Boxscores(datetime(year, month, day))
    all_games = games.games[str(month)+'-'+str(day)+'-'+str(year)]
    
    for game in all_games:
        home_teams.append(game['home_name'])
        away_teams.append(game['away_name'])
        home_scores.append(game['home_score'])
        away_scores.append(game['away_score'])
    return home_teams,away_teams,home_scores,away_scores
    

In [8]:
year,month,day = get_yesterday()
print(year, month, day)

2020 9 22


In [9]:
type(year)

int

In [12]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 12,13)

In [13]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i])

In [7]:
kp = pd.read_csv('kenpom.csv')
kp.head()

,team,off,def,tempo,luck
0,Kansas,115.8,85.5,67.3,0.040
1,Gonzaga,121.3,94.4,71.9,0.050
2,Baylor,113.5,88.1,66.2,0.016
3,Dayton,119.1,94.1,67.6,0.002
4,Duke,115.7,91.1,72.0,-0.009


In [8]:
for index, row in kp.iterrows():
    name = row['team']
    name_list = name.split()
    if name_list[-1] == "St.":
        name_list[-1] = "State"
        new_name = " ".join(str(x) for x in name_list)
        kp['team'][index] = new_name
        
        
    

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Python36_64\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [9]:
teams_ken_pom = kp['team'].tolist()

In [10]:
teams = Teams()
teams_sportsreference = []
for team in teams:
    teams_sportsreference.append(team.name)

In [11]:
for team in teams_sportsreference:
    if team not in teams_ken_pom:
        print(team)

Alabama-Birmingham
Albany (NY)
Arkansas-Pine Bluff
Bethune-Cookman
Bowling Green State
Brigham Young
Cal State Bakersfield
Cal State Fullerton
Cal State Northridge
California Baptist
UC-Davis
UC-Irvine
UC-Riverside
UC-Santa Barbara
University of California
Central Connecticut State
Central Florida
Citadel
College of Charleston
Detroit Mercy
Florida International
Gardner-Webb
Grambling
Illinois-Chicago
Purdue-Fort Wayne
Cal State Long Beach
Long Island University
Louisiana-Monroe
Louisiana State
Loyola (IL)
Loyola (MD)
Maryland-Baltimore County
Maryland-Eastern Shore
Massachusetts-Lowell
Miami (FL)
Miami (OH)
Missouri-Kansas City
Omaha
Nevada-Las Vegas
North Carolina-Asheville
North Carolina-Greensboro
North Carolina State
North Carolina-Wilmington
Pennsylvania
Prairie View
Saint Francis (PA)
Saint Mary's (CA)
South Carolina Upstate
Southern California
Southern Methodist
Southern Mississippi
St. Francis (NY)
St. John's (NY)
Tennessee-Martin
Texas A&M-Corpus Christi
Texas-Arlington
Texas

In [82]:
map_names_not_same = {
                     }

In [88]:
home_teams,away_teams,home_scores,away_scores = get_scores(2020, 3 ,8)

In [89]:
h_off = []
h_def = []
h_tempo = []
h_luck = []
a_off = []
a_def = []
a_tempo = []
a_luck = []
not_in_h = []
not_in_a = []
for i,team in enumerate(home_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team:
            h_off.append(row['off'])
            h_def.append(row['def'])
            h_tempo.append(row['tempo'])
            h_luck.append(row['luck'])
            check = True
    if not check:
        h_off.append(None)
        h_def.append(None)
        h_tempo.append(None)
        h_luck.append(None)
        not_in_h.append(i)

for i,team in enumerate(away_teams):
    check = False
    for index, row in kp.iterrows():
        if row['team'] == team and i not in not_in_h:
            a_off.append(row['off'])
            a_def.append(row['def'])
            a_tempo.append(row['tempo'])
            a_luck.append(row['luck'])
            check = True
    if not check:
        not_in_a.append(i)
x = 0
for i in not_in_a:
    num = i - x
    del h_off[num]
    del h_def[num]
    del h_tempo[num]
    del h_luck[num]
    del away_teams[num]
    del home_teams[num]
    x += 1
    

In [90]:
len(h_off)

17

In [91]:
len(home_teams)

17

In [93]:
for i, val in enumerate(home_teams):
    print(home_teams[i],away_teams[i], h_off[i], a_off[i], i + 1)

Maryland Michigan 113.5 113.2 1
Michigan State Ohio State 115.2 114.3 2
Illinois Iowa 110.9 117.3 3
Houston Memphis 112.7 100.6 4
Boston University Bucknell 103.3 96.2 5
Hofstra Drexel 108.7 100.0 6
UCF East Carolina 103.5 99.6 7
William & Mary Elon 103.3 100.6 8
Winthrop Hampton 104.7 104.8 9
Colgate Lafayette 107.2 100.6 10
Liberty Lipscomb 106.1 101.2 11
Minnesota Nebraska 112.1 102.0 12
South Dakota North Dakota 108.6 102.2 13
Towson Northeastern 105.7 105.4 14
Wichita State Tulsa 105.6 103.2 15
Bradley Valparaiso 106.8 104.5 16
Chattanooga Wofford 106.6 104.5 17


In [64]:
for i, val in enumerate(home_teams):
    print(home_scores[i],home_teams[i],away_teams[i],away_scores[i], i)

83 Maryland Michigan 70 0
80 Michigan State Ohio State 69 1
78 Illinois Iowa 76 2
64 Houston Memphis 57 3
64 Boston University Bucknell 61 4
76 Tulane UConn 80 5
67 College of Charleston Delaware 79 6
61 Hofstra Drexel 43 7
94 UCF East Carolina 62 8
63 William & Mary Elon 68 9
76 Winthrop Hampton 68 10
89 Colgate Lafayette 64 11
73 Liberty Lipscomb 57 12
107 Minnesota Nebraska 75 13
79 Oral Roberts Omaha 52 14
71 South Dakota North Dakota 74 15
62 Towson Northeastern 72 16
79 Wichita State Tulsa 57 17
80 Bradley Valparaiso 66 18
97 ETSU Western Carolina 75 19
70 Chattanooga Wofford 72 20
